In [1]:
import pandas as pd
data = pd.read_csv("/home/caro/Documents/curso_geopython/Final-exercise/data/2315676.txt",delim_whitespace=True, 
                   na_values=[-9999], skiprows = [1],low_memory=False)

In [14]:
data

,STATION,STATION_NAME,DATE,TAVG,TMAX,TMIN,TAVG2,TAVG2_C,TMAX_C,TMIN_C,YEAR,MO,SEAS
0,GHCND:FI000007501,SODANKYLA-AWS-FI,19080101,NaN,2.0,-37.0,-17.5,-27.5,-16.7,-38.3,1908,01,WI
1,GHCND:FI000007501,SODANKYLA-AWS-FI,19080102,NaN,6.0,-26.0,-10.0,-23.3,-14.4,-32.2,1908,01,WI
2,GHCND:FI000007501,SODANKYLA-AWS-FI,19080103,NaN,7.0,-27.0,-10.0,-23.3,-13.9,-32.8,1908,01,WI
3,GHCND:FI000007501,SODANKYLA-AWS-FI,19080104,NaN,-3.0,-27.0,-15.0,-26.1,-19.4,-32.8,1908,01,WI
4,GHCND:FI000007501,SODANKYLA-AWS-FI,19080105,NaN,4.0,-36.0,-16.0,-26.7,-15.6,-37.8,1908,01,WI
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41060,GHCND:FI000007501,SODANKYLA-AWS-FI,20201003,47.0,51.0,NaN,47.0,8.3,10.6,NaN,2020,10,AU
41061,GHCND:FI000007501,SODANKYLA-AWS-FI,20201004,43.0,47.0,37.0,43.0,6.1,8.3,2.8,2020,10,AU
41062,GHCND:FI000007501,SODANKYLA-AWS-FI,20201005,42.0,NaN,37.0,42.0,5.6,NaN,2.8,2020,10,AU
41063,GHCND:FI000007501,SODANKYLA-AWS-FI,20201006,45.0,46.0,43.0,45.0,7.2,7.8,6.1,2020,10,AU


In [3]:
## item 2

In [4]:
nulls = data["TAVG"].isna()
data["TAVG2"]=(data["TMAX"]+data["TMIN"])/2
for i, row in data.iterrows():
    if nulls[i]==False:
        #print(i)
        #print(nulls[i])
        data.at[i,"TAVG2"]=data.at[i,"TAVG"]

In [5]:
## item 3

In [6]:
def fahr_to_celsius(temp_fahrenheit):
    return (temp_fahrenheit - 32)/1.8

In [7]:
data["TAVG2_C"]=round(data["TAVG2"].apply(fahr_to_celsius),1)

In [8]:
data["TMAX_C"]=round(data["TMAX"].apply(fahr_to_celsius),1)

In [9]:
data["TMIN_C"]=round(data["TMIN"].apply(fahr_to_celsius),1)

In [10]:
## item 4

In [11]:
data["YEAR"] = data["DATE"].astype(str).str.slice(start=0,stop=4)

In [12]:
data["MO"] = data["DATE"].astype(str).str.slice(start=4,stop=6)

In [13]:
for i, row in data.iterrows():
    if data.at[i,"MO"]=='12' or data.at[i,"MO"]=='01' or data.at[i,"MO"]== '02':
        data.at[i,"SEAS"]='WI'
    elif data.at[i,"MO"]=='03' or data.at[i,"MO"]=='04' or data.at[i,"MO"]== '05':
        data.at[i,"SEAS"]='SP'
    elif data.at[i,"MO"]=='06' or data.at[i,"MO"]=='07' or data.at[i,"MO"]== '08':
        data.at[i,"SEAS"]='SU'
    elif data.at[i,"MO"]=='09' or data.at[i,"MO"]=='10' or data.at[i,"MO"]== '11':
        data.at[i,"SEAS"]='AU'

In [41]:
grouped = data.groupby(['SEAS', 'YEAR'])
winter = grouped.get_group("WI")[['TAVG2_C']]

In [56]:
grouped = data.groupby(['YEAR', 'SEAS'])['TAVG2_C'].mean()

In [57]:
print(grouped)

YEAR  SEAS
1908  AU      -1.857955
      SP      -2.350575
      SU      11.427778
      WI     -12.325287
1909  AU      -0.156098
                ...    
2019  WI     -11.395556
2020  AU       7.716216
      SP      -0.827174
      SU      14.437363
      WI      -9.195000
Name: TAVG2_C, Length: 452, dtype: float64


In [67]:
data["YEAR"]=data["YEAR"].astype(int)
data.dtypes
interval = data.loc[(data["YEAR"]>=1951)&(data["YEAR"]<=1980)]
seasonal_data_5180 = interval.groupby("SEAS")["TAVG2_C"].mean().to_frame(name='ref_temp')

In [87]:
seasonal_data_5180

,ref_temp
SEAS,
AU,-0.814731
SP,-2.338202
SU,12.400217
WI,-13.553102


In [85]:
type(grouped)
grouped.to_frame("seasonal_data")
type(seasonal_data)

pandas.core.series.Series

In [84]:
seasonal_data=seasonal_data.merge(seasonal_data_5180, on='SEAS', how = 'left')

AttributeError: 'Series' object has no attribute 'merge'